In [1]:

!pip install google-generativeai python-dotenv pandas tqdm pillow


  Using cached pyasn1_modules-0.4.2-py3-none-any.whl.metadata (3.5 kB)
  Using cached grpcio-1.71.0-cp313-cp313-win_amd64.whl.metadata (4.0 kB)
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------------------------------- 1.3/1.3 MB 8.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/13.3 MB ? eta -:--:--
   ---- ----------------------------------- 1.6/13.3 MB 8.6 MB/s eta 0:00:02
   --------- ------------------------------ 3.1/13.3 MB 7.6 MB/s eta 0:00:02
   -------------- ------------------------- 4.7/13.3 MB 7.6 MB/s eta 0:00:02
   ------------------- -------------------- 6.6/13.3 MB 8.1 MB/s eta 0:00:01
   ----------------------- ---------------- 7.9/13.3 MB 7.6 MB/s eta 0:00:01
   ----------------------------- ---------- 9.7/13.3 MB 7.9 MB/s eta 0:00:01
   ----------------------------------- ---- 11.8/13.3 MB 8.2 MB/s eta 0:00:01
   ---------------------------------------- 13.3/13.3 MB 8.1 MB/s eta 0:00:00
Using cached pyasn1_mod


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import time
import json
import pandas as pd
import google.generativeai as genai
from dotenv import load_dotenv
from tqdm import tqdm
from PIL import Image


In [3]:

load_dotenv()

API_KEY = os.getenv('GEMINI_API_KEY')

genai.configure(api_key=API_KEY)


In [4]:

model = genai.GenerativeModel('gemini-1.5-flash')


In [5]:
import base64
from PIL import Image
import io

def load_image_for_gemini(image_path):
    """Load image and prepare it for Gemini API."""
    with open(image_path, "rb") as f:
        image_data = f.read()
    return image_data


In [ ]:
import time
import pandas as pd
from tqdm import tqdm


image_folder = r'C:\Users\47770057\Desktop\Final Attempt\Anime Images'
image_paths = [os.path.join(image_folder, img) for img in os.listdir(image_folder)]


dataset = []


def query_gemini(image_bytes):
    """Query Gemini with an image."""
    response = model.generate_content(
        contents=[
            {
                "mime_type": "image/jpeg",
                "data": base64.b64encode(image_bytes).decode("utf-8"),
            },
            {
                "text": """Analyze the image.

If it contains a SINGLE character, respond strictly in the following JSON format only (no extra explanation):

{
	"Age": "Young Adult",
	"Gender": "Female",
	"Ethnicity": "Asian",
	"Hair Style": "Ponytail",
	"Hair Color": "Black",
	"Hair Length": "Long",
	"Eye Color": "Brown",
	"Body Type": "Slim",
	"Dress": "Green Yukata"
}

If the image contains MULTIPLE characters, respond only with: too many characters  
If it contains NO characters, respond only with: no characters"""
            }
        ],
        stream=False
    )
    return response.text.strip()

# Start processing
for img_path in tqdm(image_paths):
    try:
        # Load image
        img_bytes = load_image_for_gemini(img_path)
        
        # Query Gemini
        image_details = query_gemini(img_bytes)
        
        # Append to dataset
        dataset.append({
            "image": os.path.basename(img_path),
            "image_details": image_details
        })
        
        # Wait for 5 seconds
        time.sleep(5)

    except Exception as e:
        print(f"Error processing {img_path}: {str(e)}")
        continue


df = pd.DataFrame(dataset)


100%|██████████| 122/122 [12:28<00:00,  6.14s/it]


In [ ]:

df.to_csv('synthetic_image_dataset.csv', index=False)


df.to_json('synthetic_image_dataset.jsonl', orient='records', lines=True)

print("Dataset saved successfully!")


Dataset saved successfully!


In [ ]:
from IPython.display import FileLink


FileLink('synthetic_image_dataset.csv')


c:\Users\47770057\Desktop\Final Attempt\synthetic_image_dataset.csv

In [ ]:
import pandas as pd
import re
import json

# Clean the dataset
def clean_image_details(detail):
    if not isinstance(detail, str):
        return "invalid format"
    
    detail = detail.strip()

    # Remove the ```json or similar formatting markers
    detail = re.sub(r'```json|```|~~~json|~~~', '', detail).strip()

    # Try parsing to ensure valid JSON and fix double quotes
    try:
        # Replace double double-quotes with single double-quotes
        detail = re.sub(r'""([^"{}:,]+)""', r'"\1"', detail)
        detail = re.sub(r"'{2}([^'{}:,]+)'{2}", r'"\1"', detail)

        # Convert the cleaned string to a JSON object and back to formatted string
        parsed = json.loads(detail)
        return json.dumps(parsed, indent=4)
    except Exception as e:
        return "invalid format"


df = pd.read_csv("synthetic_image_dataset.csv")


df["image_details"] = df["image_details"].apply(clean_image_details)


df = df[df["image_details"] != "invalid format"]


df.to_csv("cleaned_synthetic_image_dataset.csv", index=False)
print("Cleaned dataset saved as 'cleaned_synthetic_image_dataset.csv'")


Cleaned dataset saved as 'cleaned_synthetic_image_dataset.csv'


In [22]:
!pip install pandas opencv-python



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import pandas as pd
import cv2
import os
import uuid
from tqdm import tqdm


image_folder = r"C:\Users\47770057\Desktop\Final Attempt\Anime Images"
input_csv = r"C:\Users\47770057\Desktop\Final Attempt\cleaned_synthetic_image_dataset.csv"
output_csv = "augmented_synthetic_image_dataset.csv"


df = pd.read_csv("cleaned_synthetic_image_dataset.csv")


augmented_data = []

# Define simple augmentations using OpenCV
def augment_image(image):
    return [
        cv2.flip(image, 1),  # Horizontal flip
        cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE),  # Rotate 90 deg
        cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE),  # Rotate -90 deg
        cv2.GaussianBlur(image, (5, 5), 0),  # Blurring
        cv2.convertScaleAbs(image, alpha=1.2, beta=30)  # Brightness/contrast
    ]

# Iterate and augment
for _, row in tqdm(df.iterrows(), total=len(df)):
    original_image = row["image"]
    image_path = os.path.join(image_folder, original_image)

    if not os.path.isfile(image_path):
        print(f"Image not found: {image_path}")
        continue

    img = cv2.imread(image_path)

    if img is None:
        print(f"Failed to read: {image_path}")
        continue

    # Perform augmentations
    augmented_imgs = augment_image(img)

    for idx, aug_img in enumerate(augmented_imgs):
        new_filename = f"{os.path.splitext(original_image)[0]}_aug{idx+1}.jpg"
        new_path = os.path.join(image_folder, new_filename)
        cv2.imwrite(new_path, aug_img)

        # Add new entry to augmented data
        augmented_data.append({
            "image": new_filename,
            "image_details": row["image_details"]
        })


augmented_df = pd.DataFrame(augmented_data)

# Combine original and augmented datasets
final_df = pd.concat([df, augmented_df], ignore_index=True)

# Save to new CSV
final_df.to_csv(output_csv, index=False)
print(f"Augmented dataset saved to {output_csv}")


100%|██████████| 84/84 [00:01<00:00, 79.79it/s]

Augmented dataset saved to augmented_synthetic_image_dataset.csv
